In [7]:
import numpy as np
import pandas as pd
import networkx as nx 

In [35]:
np.random.seed(1)

We create the graphs for each model

In [9]:
G_complete = nx.complete_graph(1000)
G_BA = nx.barabasi_albert_graph(1000, m = 5, seed = 1)
G_WS = nx.watts_strogatz_graph(1000, k = 4, p = 0.2, seed = 1)
# G_tree = nx.tree_graph(1000, seed = 1)
# G_star = nx.star_graph(1000, seed = 1)
# G_lattice = nx.latt
G_ER = nx.erdos_renyi_graph(1000, p = 0.5, seed = 1)




In [10]:
nx.set_node_attributes(G_complete, "Susceptible", "Status")
nx.set_node_attributes(G_BA, "Susceptible", "Status")
nx.set_node_attributes(G_WS, "Susceptible", "Status")
nx.set_node_attributes(G_ER, "Susceptible", "Status")

In [34]:
p_0 = 0.01
# population_array = np.arange(1000)
infected_0 = np.random.choice(1000, int(p_0*1000), replace=False)

In [36]:
infected_0

array([827, 603, 335, 420, 666, 989,  87, 459, 359, 392])

In [24]:
# G_complete.nodes(data =True)
# G_BA.nodes(data =True)
# G_WS.nodes(data =True)
# G_ER.nodes(data =True)


We set the infected people at time 0

In [43]:
for i in infected_0 : 
    G_complete.nodes[i]["Status"] = "Infected"
    G_BA.nodes[i]["Status"] = "Infected"
    G_WS.nodes[i]["Status"] = "Infected"
    G_ER.nodes[i]["Status"] = "Infected"

In [44]:
infected_neightbour_i = len([v for v in nx.neighbors(G_complete, 0) if G_complete.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_BA, 0) if G_BA.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_WS, 0) if G_BA.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_ER, 0) if G_BA.nodes[v]["Status"] == "Infected"])

In [45]:
infected_neightbour_i

10

In [46]:
for node, attribute in nx.get_node_attributes(G_complete, "Status").items() : 
    if attribute == "Infected" : 
        

{0: 'Susceptible',
 1: 'Susceptible',
 2: 'Susceptible',
 3: 'Susceptible',
 4: 'Susceptible',
 5: 'Susceptible',
 6: 'Susceptible',
 7: 'Susceptible',
 8: 'Susceptible',
 9: 'Susceptible',
 10: 'Susceptible',
 11: 'Susceptible',
 12: 'Susceptible',
 13: 'Susceptible',
 14: 'Susceptible',
 15: 'Susceptible',
 16: 'Susceptible',
 17: 'Susceptible',
 18: 'Susceptible',
 19: 'Susceptible',
 20: 'Susceptible',
 21: 'Susceptible',
 22: 'Susceptible',
 23: 'Susceptible',
 24: 'Susceptible',
 25: 'Susceptible',
 26: 'Susceptible',
 27: 'Susceptible',
 28: 'Susceptible',
 29: 'Susceptible',
 30: 'Susceptible',
 31: 'Susceptible',
 32: 'Susceptible',
 33: 'Susceptible',
 34: 'Susceptible',
 35: 'Susceptible',
 36: 'Susceptible',
 37: 'Susceptible',
 38: 'Susceptible',
 39: 'Susceptible',
 40: 'Susceptible',
 41: 'Susceptible',
 42: 'Susceptible',
 43: 'Susceptible',
 44: 'Susceptible',
 45: 'Susceptible',
 46: 'Susceptible',
 47: 'Susceptible',
 48: 'Susceptible',
 49: 'Susceptible',
 50: 'Susc